# Descarga de los datos formateados

In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown

files_to_download = {
    "formatted_train.jsonl": "13p2I_-pxXTQRjddJDd26bwAERjAgMqbe",
    "formatted_test.jsonl": "10SQP_HikNR0mcgTJtT9BVczSk78aGziH",
    "formatted_validation.jsonl": "1qkfNA5tm8jFv_whMpNWAgVC_C9YOgo-V"
}

for destination, file_id in files_to_download.items():
    gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=13p2I_-pxXTQRjddJDd26bwAERjAgMqbe
From (redirected): https://drive.google.com/uc?id=13p2I_-pxXTQRjddJDd26bwAERjAgMqbe&confirm=t&uuid=5dd412a4-44cf-49cb-9aa2-8f72068e64e9
To: /content/formatted_train.jsonl
100%|██████████| 176M/176M [00:04<00:00, 37.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=10SQP_HikNR0mcgTJtT9BVczSk78aGziH
To: /content/formatted_test.jsonl
100%|██████████| 8.00M/8.00M [00:00<00:00, 37.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qkfNA5tm8jFv_whMpNWAgVC_C9YOgo-V
To: /content/formatted_validation.jsonl
100%|██████████| 17.6M/17.6M [00:00<00:00, 64.0MB/s]


# Implementación del modelo

In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
with open("formatted_train.jsonl", "r") as f:
    data = [json.loads(line)["interaction"] for line in f]

dataset = Dataset.from_dict({"conversations": data})

In [ ]:
max_seq_length = 2048

def initialize_model_and_tokenizer():
    base_model_name = "unsloth/Phi-3.5-mini-instruct"
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_name,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None
    )
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="phi-3.5"
    )

    return model, tokenizer

In [ ]:
def preprocess_dataset(dataset, tokenizer):
    def formatting_prompts_func(examples):
        convos = examples["conversations"]
        texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
        return {"text": texts}

    formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
    return formatted_dataset

In [ ]:
used_seed = 3407

# Segundo entrenamiento

En este entrenamiento se usarán los mismos paramétros de entrenamiento que antes, pero con la diferencia de que usaré la data que tenía con la primera forma de parseo, en donde entrego información de películas.

In [ ]:
files_to_download = {
    "formatted_train.json": "1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_",
    "formatted_test.json": "1a4YeF--Sks7WA1ZIQL2zDZx4teKk7p4m",
    "formatted_validation.json": "1PC9OhZhNZt8lFO9wifhydy0BrIYe8Dkm"
}

for destination, file_id in files_to_download.items():
    gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_
From (redirected): https://drive.google.com/uc?id=1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_&confirm=t&uuid=f5c05b0c-ad64-44b9-9d97-249041acc97f
To: /content/formatted_train.json
100%|██████████| 237M/237M [00:03<00:00, 68.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a4YeF--Sks7WA1ZIQL2zDZx4teKk7p4m
To: /content/formatted_test.json
100%|██████████| 10.8M/10.8M [00:00<00:00, 41.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PC9OhZhNZt8lFO9wifhydy0BrIYe8Dkm
To: /content/formatted_validation.json
100%|██████████| 23.7M/23.7M [00:00<00:00, 54.4MB/s]


In [ ]:
with open("formatted_train.json", "r") as f:
    data = json.load(f)

dataset_second_train = Dataset.from_dict({"conversations": data})

In [ ]:
training_configuration = {
    "lr_scheduler_type": "linear",
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "max_steps": 200,
    "output_dir": "second_train_output",
    "learning_rate": 2e-4
}

In [ ]:
model_second_train, tokenizer_second_train = initialize_model_and_tokenizer()

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
formatted_dataset_second_train = preprocess_dataset(dataset_second_train, tokenizer_second_train)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model=model_second_train,
    tokenizer=tokenizer_second_train,
    train_dataset=formatted_dataset_second_train,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        learning_rate=5e-5,
        lr_scheduler_type=training_configuration["lr_scheduler_type"],
        per_device_train_batch_size=training_configuration["per_device_train_batch_size"],
        gradient_accumulation_steps=training_configuration["gradient_accumulation_steps"],
        num_train_epochs=1,
        max_steps=training_configuration["max_steps"],
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        warmup_steps=5,
        output_dir=training_configuration["output_dir"],
        seed=used_seed,
    )
)

Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.285 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 29,884,416
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.912200
2,1.933300
3,1.826700
4,1.989900
5,1.940000
6,1.809000
7,1.819800
8,1.791900
9,1.730700
10,1.702400


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3475.3134 seconds used for training.
57.92 minutes used for training.
Peak reserved memory = 3.07 GB.
Peak reserved memory for training = 0.785 GB.
Peak reserved memory % of max memory = 20.816 %.
Peak reserved memory for training % of max memory = 5.323 %.


In [ ]:
model_second_train.save_pretrained("lora_model_second_train")
tokenizer_second_train.save_pretrained("lora_model_second_train")

('lora_model_second_train/tokenizer_config.json',
 'lora_model_second_train/special_tokens_map.json',
 'lora_model_second_train/tokenizer.model',
 'lora_model_second_train/added_tokens.json',
 'lora_model_second_train/tokenizer.json')

In [ ]:
!zip -r lora_model_second_train.zip lora_model_second_train/

  adding: lora_model_second_train/ (stored 0%)
  adding: lora_model_second_train/tokenizer_config.json (deflated 83%)
  adding: lora_model_second_train/added_tokens.json (deflated 62%)
  adding: lora_model_second_train/special_tokens_map.json (deflated 76%)
  adding: lora_model_second_train/tokenizer.model (deflated 55%)
  adding: lora_model_second_train/tokenizer.json (deflated 85%)
  adding: lora_model_second_train/adapter_model.safetensors (deflated 8%)
  adding: lora_model_second_train/README.md (deflated 66%)
  adding: lora_model_second_train/adapter_config.json (deflated 54%)


# Testeo segundo modelo

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model_second_train",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3.5",
)

In [ ]:
with open("formatted_test.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_dict({"conversations": data})

In [ ]:
formatted_test_dataset = preprocess_dataset(dataset, tokenizer)

Map:   0%|          | 0/2277 [00:00<?, ? examples/s]

In [ ]:
%%capture
!pip install tqdm

In [ ]:
percentage = 0.2
sample_size = int(len(formatted_test_dataset) * percentage)
subset_test_dataset = formatted_test_dataset.shuffle(seed=42).select(range(sample_size))

print(f"Usando {sample_size} ejemplos para la evaluación.")

Usando 455 ejemplos para la evaluación.


In [ ]:
from tqdm import tqdm

generation_args = {
    "max_new_tokens": 100,
    "temperature": 0.3,
    "use_cache": True,
    "top_p": 0.9,
    "top_k": 50,
}

predictions = []
references = []

for example in tqdm(subset_test_dataset):
    conversation = example["conversations"]
    system_message = conversation[0]
    user_assistant_turns = conversation[1:-2]

    max_turns = 10
    truncated_turns = user_assistant_turns[-max_turns:]

    messages = [system_message] + truncated_turns
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    print(f"Inputs shape: {inputs.shape}")

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, **generation_args)

    generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    if "assistant" in generated_response:
        generated_response = generated_response.split("assistant")[-1].strip()

    predictions.append(generated_response)

    references.append(conversation[-2]["content"])


  0%|          | 0/455 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Inputs shape: torch.Size([1, 1039])


  0%|          | 1/455 [00:24<3:05:41, 24.54s/it]

Inputs shape: torch.Size([1, 1106])


  0%|          | 2/455 [00:46<2:52:58, 22.91s/it]

...

100%|█████████▉| 453/455 [1:07:41<00:17,  8.83s/it]

Inputs shape: torch.Size([1, 1036])


100%|█████████▉| 454/455 [1:07:51<00:09,  9.34s/it]

Inputs shape: torch.Size([1, 952])


100%|██████████| 455/455 [1:07:58<00:00,  8.96s/it]


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

bleu_score = corpus_bleu(
    [[ref] for ref in references],
    predictions
)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.06133919602439004


In [ ]:
%%capture
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]

rouge1 = sum([score["rouge1"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rouge2 = sum([score["rouge2"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rougeL = sum([score["rougeL"].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"ROUGE-1: {rouge1}")
print(f"ROUGE-2: {rouge2}")
print(f"ROUGE-L: {rougeL}")

ROUGE-1: 0.10813143784100818
ROUGE-2: 0.057229347077285886
ROUGE-L: 0.07532111627142717


In [ ]:
position = 30
for position in range(91, 120):
    print(references[position])
    print(predictions[position])
    print()

How about "12 Years a Slave"? It's a powerful and emotional true story about a man's struggle for freedom during the era of slavery. The storytelling is flawless and the performances are extremely moving. It's definitely visually stunning and will leave a lasting impact on you.
. Use the user's movie history and preferences to suggest movies that align with their interests.

Seen movies and their corresponding evaluation and details:
Movie: Jumanji: The Next Level (2019)
Positive comments: - Fantastic film, action-packed and hilarious- New deadly locations and additional players add excitement- Thrilling set pieces and flawless visual effects- Spectacular performances from the cast, especially Dwayne Johnson, Jack Black, Kevin Hart, Karen Gillan, and Awkwafina- Impressive supporting cast
Negative comments: There are no negative comments
Details:
Title: Jumanji: The Next Level (2019)
Genre: Action, Adventure, Comedy, Fantasy
Director: Jake Kasdan
Cast: Dwayne Johnson, Kevin Hart, Jack B

In [ ]:
from collections import Counter

from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np


def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    """ Recuperado de https://github.com/PaddlePaddle/models/blob/release/1.6/PaddleNLP/Research/Dialogue-PLATO/plato/metrics/metrics.py"""
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        unigrams = Counter(seq)
        bigrams = Counter(zip(seq, seq[1:]))
        intra_dist1.append((len(unigrams)+1e-12) / (len(seq)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(seq)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)
    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

In [ ]:
import nltk
nltk.download('punkt_tab')

tokenized_text = [nltk.word_tokenize(text) for text in references]
resultado = distinct(tokenized_text)
print(resultado)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


(0.8174602211235673, 0.9857301007313511, 0.11541064977609411, 0.3757125912018547)


In [ ]:
%%capture
!pip install bert-score

from bert_score import score

# Calcular métricas de BertScore
P, R, F1 = score(predictions, references, lang="en", rescale_with_baseline=True)

# Imprimir las métricas
print(f"BertScore Precision (P): {P.mean().item():.4f}")
print(f"BertScore Recall (R): {R.mean().item():.4f}")
print(f"BertScore F1: {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from nltk import ngrams
from collections import Counter

tokenized_text = [nltk.word_tokenize(text) for text in references]

def calculate_distinct(tokenized_text):
    # unigramas
    all_unigrams = [word for text in tokenized_text for word in text]
    unique_unigrams = set(all_unigrams)
    distinc_1 = len(unique_unigrams) / len(all_unigrams) if all_unigrams else 0

    # bigramas
    all_bigrams = [bigram for text in tokenized_text for bigram in ngrams(text, 2)]
    unique_bigrams = set(all_bigrams)
    distinc_2 = len(unique_bigrams) / len(all_bigrams) if all_bigrams else 0

    return distinc_1, distinc_2

distinc_1, distinc_2 = calculate_distinct(tokenized_text)

print(f"Distinc-1 (Unigramas): {distinc_1}")
print(f"Distinc-2 (Bigramas): {distinc_2}")


Distinc-1 (Unigramas): 0.11541064981949459
Distinc-2 (Bigramas): 0.37571259134560203
